# Koululaisten ruokatottumukset - Analyysi

Analysoidaan 20 000 rivin dataset koululaisten ruokatottumuksista eri maissa.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Asetetaan visualisointien tyyli
plt.style.use('default')
sns.set_palette('husl')

In [ ]:
# Ladataan data
df = pd.read_csv('./assets/Data/13100194.csv', sep=';', encoding='utf-8')

print(f"Dataset koko: {df.shape}")
print(f"Rivejä: {len(df)}, Sarakkeita: {len(df.columns)}")
print("\nSarakkeet:")
for col in df.columns:
    print(f"- {col}")

In [ ]:
# Tutkitaan datan rakennetta
print("Ensimmäiset 5 riviä:")
df.head()

In [ ]:
# Tutkitaan eri kategorioita
print("Maat/alueet:")
print(df['GÉO'].unique()[:10])  # Ensimmäiset 10

print("\nSukupuolet:")
print(df['Sexe'].unique())

print("\nIkäryhmät:")
print(df['Groupe d\'âge'].unique())

print("\nRuokatottumukset (aktiviteetit):")
print(df['Activité'].unique())

In [ ]:
# Puhdistetaan data
# Poistetaan rivit joissa ei ole arvoa
df_clean = df.dropna(subset=['VALEUR'])

# Muutetaan VALEUR numeeriseksi
df_clean['VALEUR'] = pd.to_numeric(df_clean['VALEUR'], errors='coerce')

# Poistetaan rivit joissa VALEUR ei ole numero
df_clean = df_clean.dropna(subset=['VALEUR'])

print(f"Puhdistettu data: {df_clean.shape[0]} riviä")
print(f"Keskiarvo: {df_clean['VALEUR'].mean():.1f}%")
print(f"Mediaani: {df_clean['VALEUR'].median():.1f}%")

In [ ]:
# Visualisointi 1: Sukupuolierot ruokatottumuksissa
sukupuoli_data = df_clean.groupby(['Sexe', 'Activité'])['VALEUR'].mean().reset_index()

fig = px.bar(sukupuoli_data, 
             x='Activité', 
             y='VALEUR', 
             color='Sexe',
             title='Sukupuolierot ruokatottumuksissa (keskiarvo %)',
             labels={'VALEUR': 'Prosentti (%)', 'Activité': 'Ruokatottumukset'})

fig.update_layout(xaxis_tickangle=-45, height=600)
fig.show()

In [ ]:
# Visualisointi 2: Ikäryhmien väliset erot
ika_data = df_clean.groupby(['Groupe d\'âge', 'Activité'])['VALEUR'].mean().reset_index()

fig = px.line(ika_data, 
              x='Groupe d\'âge', 
              y='VALEUR', 
              color='Activité',
              title='Ruokatottumukset ikäryhmittäin',
              labels={'VALEUR': 'Prosentti (%)', 'Groupe d\'âge': 'Ikäryhmä'})

fig.update_layout(height=600)
fig.show()

In [ ]:
# Visualisointi 3: Maiden vertailu (top 10 maata)
maa_data = df_clean.groupby('GÉO')['VALEUR'].mean().sort_values(ascending=False).head(10)

fig = px.bar(x=maa_data.index, 
             y=maa_data.values,
             title='Top 10 maata - Parhaat ruokatottumukset (keskiarvo %)',
             labels={'x': 'Maa', 'y': 'Keskiarvo (%)'})

fig.update_layout(xaxis_tickangle=-45, height=600)
fig.show()

In [ ]:
# Visualisointi 4: Heatmap - Maat vs Ruokatottumukset
# Otetaan vain tärkeimmät maat ja aktiviteetit
top_maat = df_clean.groupby('GÉO')['VALEUR'].count().nlargest(15).index
heatmap_data = df_clean[df_clean['GÉO'].isin(top_maat)]

pivot_data = heatmap_data.groupby(['GÉO', 'Activité'])['VALEUR'].mean().unstack(fill_value=0)

plt.figure(figsize=(12, 8))
sns.heatmap(pivot_data, annot=True, fmt='.1f', cmap='YlOrRd')
plt.title('Ruokatottumukset maittain - Heatmap (%)')
plt.xlabel('Ruokatottumukset')
plt.ylabel('Maat')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Visualisointi 5: Frekvenssianalyysi (kuinka usein syödään)
freq_data = df_clean.groupby('Fréquence')['VALEUR'].mean().sort_values(ascending=False)

fig = px.pie(values=freq_data.values, 
             names=freq_data.index,
             title='Ruokailutiheys - Jakauma')

fig.show()

In [ ]:
# Yhteenveto ja tilastot
print("DATASET YHTEENVETO:")
print("="*50)
print(f"Kokonaisrivejä: {len(df_clean)}")
print(f"Maita: {df_clean['GÉO'].nunique()}")
print(f"Ruokatottumuksia: {df_clean['Activité'].nunique()}")
print(f"Ikäryhmiä: {df_clean['Groupe d\'âge'].nunique()}")

print("\nKESKEISET LÖYDÖKSET:")
print("="*50)

# Sukupuolierot
sukupuoli_keskiarvo = df_clean.groupby('Sexe')['VALEUR'].mean()
print(f"Sukupuolierot (keskiarvo):")
for sukupuoli, arvo in sukupuoli_keskiarvo.items():
    print(f"  {sukupuoli}: {arvo:.1f}%")

# Paras ja huonoin maa
maa_keskiarvo = df_clean.groupby('GÉO')['VALEUR'].mean()
print(f"\nParas maa: {maa_keskiarvo.idxmax()} ({maa_keskiarvo.max():.1f}%)")
print(f"Huonoin maa: {maa_keskiarvo.idxmin()} ({maa_keskiarvo.min():.1f}%)")

# Yleisimmät ruokatottumukset
print(f"\nYleisimmät ruokatottumukset:")
aktiviteetti_keskiarvo = df_clean.groupby('Activité')['VALEUR'].mean().sort_values(ascending=False)
for akt, arvo in aktiviteetti_keskiarvo.head(3).items():
    print(f"  {akt}: {arvo:.1f}%")